In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

## Objective
We want to retain customers by identifying those who are about to cancel their contract with the company and send them promotional messages,<br>
offering discounts and other benefits, hoping that after receiving the benefits, they will stay with the company. On the other hand we don't want to <br>give promotions to all of our customers.

In [2]:
df = pd.read_csv('input/data.csv')
df.TotalCharges = pd.to_numeric(df.TotalCharges, errors='coerce')
df.TotalCharges = df.TotalCharges.fillna(0)
df.Churn = (df.Churn == 'Yes').astype('uint8')
df.SeniorCitizen = df.SeniorCitizen.astype('uint8')
df.tenure = df.tenure.astype('uint16') 
df.MonthlyCharges = df.MonthlyCharges.astype('float32')
df.columns = df.columns.str.lower().str.replace(' ', '_')
string_columns = list(df.dtypes[df.dtypes.values == 'object'].index)
for col in string_columns:
    df[col] = df[col].str.lower().str.replace(' ', '_')
    

In [3]:
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=11)
df_train, df_val = train_test_split(df_train_full, test_size=0.25, random_state=11)
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_train_full = df_train_full.reset_index(drop=True)
y_train_full =  df_train_full.churn.values
y_train = df_train['churn'].values
y_val = df_val['churn'].values
y_test = df_test.churn.values
del df_train['churn']
del df_val['churn']
del df_test['churn']

In [4]:
numerical = [ 'tenure', 'monthlycharges', 'totalcharges']
categorical = ['gender',
             'seniorcitizen',
             'partner',
             'dependents',
             'phoneservice',
             'multiplelines',
             'internetservice',
             'onlinesecurity',
             'onlinebackup',
             'deviceprotection',
             'techsupport',
             'streamingtv',
             'streamingmovies',
             'contract',
             'paperlessbilling',
             'paymentmethod']


In [5]:
def train(df, y, C):
    cat = df[categorical + numerical].to_dict(orient='records')
    
    dv = DictVectorizer(sparse=False)
    dv.fit(cat)
    
    X = dv.transform(cat)
    
    model = LogisticRegression(solver='liblinear', C=C)
    model.fit(X, y)
    
    return dv, model


def predict(df, dv, model):
    cat = df[categorical + numerical].to_dict(orient='records')
    
    X = dv.transform(cat)
    y_pred = model.predict_proba(X)[:,1]
    
    return y_pred


In [6]:
nfolds = 5 
kfold = KFold(n_splits=nfolds, shuffle=True, random_state=1)
for C in [0.001, 0.01, 0.1, 0.5, 1, 10]:
    aucs = []
    for train_idx, val_idx in kfold.split(df_train_full):
        df_train = df_train_full.iloc[train_idx]
        df_val = df_train_full.iloc[val_idx]

        y_train = df_train.churn.values
        y_val = df_val.churn.values

        dv, model = train(df_train, y_train, C=C)
        y_pred = predict(df_val, dv, model)

        auc = roc_auc_score(y_val, y_pred)
        aucs.append(auc)
    print(f'C= {C}, AUC mean = {np.mean(aucs):.3f}, AUC std = {np.std(aucs):.3f}')

C= 0.001, AUC mean = 0.823, AUC std = 0.016
C= 0.01, AUC mean = 0.840, AUC std = 0.012
C= 0.1, AUC mean = 0.844, AUC std = 0.012
C= 0.5, AUC mean = 0.844, AUC std = 0.012
C= 1, AUC mean = 0.844, AUC std = 0.012
C= 10, AUC mean = 0.844, AUC std = 0.012


In [7]:
dv, model = train(df_train_full, y_train_full, C=0.1)
y_pred = predict(df_test, dv, model)
auc = roc_auc_score(y_test, y_pred)
print(f'AUC = {auc:.3f}') 

AUC = 0.852


## Using the model

* Function for predicting the probability of churn for a single customer

In [8]:
def predict_single(customer, dv, model):
    X = dv.transform(customer)
    y_pred = model.predict_proba(X)[:, 1]
    return y_pred[0]

In [9]:
customer = df_test.iloc[0].to_dict()

In [10]:
customer

{'customerid': '3398-fshon',
 'gender': 'female',
 'seniorcitizen': 1,
 'partner': 'no',
 'dependents': 'no',
 'tenure': 12,
 'phoneservice': 'yes',
 'multiplelines': 'yes',
 'internetservice': 'fiber_optic',
 'onlinesecurity': 'no',
 'onlinebackup': 'yes',
 'deviceprotection': 'no',
 'techsupport': 'no',
 'streamingtv': 'yes',
 'streamingmovies': 'no',
 'contract': 'month-to-month',
 'paperlessbilling': 'yes',
 'paymentmethod': 'electronic_check',
 'monthlycharges': 91.30000305175781,
 'totalcharges': 1094.5}

In [11]:
predict_single(customer, dv, model)

0.7240885111288676

## Using pickle to save and load the model

In [12]:
import pickle

* Saving the model

In [13]:
with open ('churn-model.bin', 'wb') as f_out:
    pickle.dump((dv,model), f_out)

* Loading the model

In [14]:
with open('churn-model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

## Testing the web service

In [15]:
import requests

In [16]:
url = 'http://localhost:9696/predict'

In [17]:
response = requests.post(url, json=customer)

In [18]:
result = response.json()

In [19]:
result

{'churn': True, 'churn_probability': 0.7240881673284846}